# Mechanisms-of-Action-Prediction-baseline-lightGBM

## debug

In [24]:
debug = True

## import libraries

In [25]:
import pandas as pd
import numpy as np
import copy

import lightgbm as lgb

## load data

In [26]:
train_features = pd.read_csv("../input/lish-moa/train_features.csv")
train_targets_scored = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
train_targets_nonscored = pd.read_csv("../input/lish-moa/train_targets_nonscored.csv")
test_features = pd.read_csv("../input/lish-moa/test_features.csv")
submission = pd.read_csv("../input/lish-moa/sample_submission.csv")

## preprocess

In [27]:
prep_df = pd.concat([train_features, test_features])

In [28]:
import category_encoders as ce

object_cols = ["cp_type", "cp_dose"]
ce_oe = ce.OrdinalEncoder(cols=object_cols, handle_unknown='impute')
prep_df = ce_oe.fit_transform(prep_df)
train_features = prep_df.iloc[:len(train_features),:]
test_features = prep_df.iloc[len(train_features):,:]

## train models and prediction

In [34]:
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import KFold
FOLD_NUM = 5
#mskf = MultilabelStratifiedKFold(n_splits=FOLD_NUM, shuffle=True, random_state=42)
kf = KFold(n_splits=5, shuffle=True, random_state=0)
target_cols = [c for c in train_targets_scored.columns if c != "sig_id"]
oof = train_targets_scored.copy()
num_round = 10000

In [35]:
train_X = train_features
train_y = train_targets_scored
test_X = test_features

In [36]:
params = {
'boosting_type': 'gbdt',
'objective': 'binary',
'lambda_l1': 0.001, 
'lambda_l2': 0.001,
'num_leaves': 50, 
'max_depth': 6,
'feature_fraction': 0.4,
'subsample': 0.4, 
'min_child_samples': 10,
'learning_rate': 0.01,
'num_iterations': 100, #700
'early_stopping_rounds': 100,
'random_state': 42}

In [37]:
def run_lgbm(target_col: str):
    
    X_train = train_features.drop(["sig_id"], axis=1)
    y_train = train_targets_scored[target_col]
    X_test = test_features.drop(["sig_id"], axis=1)

    y_preds = []
    models = []
    oof_train = np.zeros((len(X_train),))

    for fold_id, (train_index, valid_index) in enumerate(kf.split(X_train)):
        X_tr = X_train.loc[train_index, :]
        X_val = X_train.loc[valid_index, :]
        y_tr = y_train[train_index]
        y_val = y_train[valid_index]

        lgb_train = lgb.Dataset(X_tr,
                                y_tr,
                                categorical_feature=object_cols)

        lgb_eval = lgb.Dataset(X_val,
                               y_val,
                               reference=lgb_train,
                               categorical_feature=object_cols)

        model = lgb.train(params,
                          lgb_train,
                          valid_sets=[lgb_train, lgb_eval],
                          verbose_eval=10,
                          num_boost_round=1000,
                          early_stopping_rounds=10)


        oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
        
        y_pred = model.predict(X_test, num_iteration=model.best_iteration)

        y_preds.append(y_pred)
        models.append(model)

    return oof_train, sum(y_preds) / len(y_preds)

In [ ]:
for target_col in target_cols:
    _oof, _preds = run_lgbm(target_col)
    oof[target_col] = _oof
    submission[target_col] = _preds

Training until validation scores don't improve for 100 rounds
[10]	training's binary_logloss: 0.00231838	valid_1's binary_logloss: 0.00686659
[20]	training's binary_logloss: 0.00178966	valid_1's binary_logloss: 0.00689145
[30]	training's binary_logloss: 0.00148197	valid_1's binary_logloss: 0.00690999
[40]	training's binary_logloss: 0.00126756	valid_1's binary_logloss: 0.00693467
[50]	training's binary_logloss: 0.00109904	valid_1's binary_logloss: 0.00696489
[60]	training's binary_logloss: 0.000963435	valid_1's binary_logloss: 0.00700063
[70]	training's binary_logloss: 0.000852911	valid_1's binary_logloss: 0.00702184
[80]	training's binary_logloss: 0.000757561	valid_1's binary_logloss: 0.00706023
[90]	training's binary_logloss: 0.000676041	valid_1's binary_logloss: 0.00710775
[100]	training's binary_logloss: 0.000605027	valid_1's binary_logloss: 0.00716349
Did not meet early stopping. Best iteration is:
[100]	training's binary_logloss: 0.000605027	valid_1's binary_logloss: 0.00716349
Tr

[50]	training's binary_logloss: 0.00135351	valid_1's binary_logloss: 0.00533226
[60]	training's binary_logloss: 0.00119753	valid_1's binary_logloss: 0.00534409
[70]	training's binary_logloss: 0.00106262	valid_1's binary_logloss: 0.00535779
[80]	training's binary_logloss: 0.000950075	valid_1's binary_logloss: 0.00538255
[90]	training's binary_logloss: 0.000852733	valid_1's binary_logloss: 0.00540493
[100]	training's binary_logloss: 0.000768415	valid_1's binary_logloss: 0.00543067
Did not meet early stopping. Best iteration is:
[100]	training's binary_logloss: 0.000768415	valid_1's binary_logloss: 0.00543067
Training until validation scores don't improve for 100 rounds
[10]	training's binary_logloss: 0.00332164	valid_1's binary_logloss: 0.00380232
[20]	training's binary_logloss: 0.00244593	valid_1's binary_logloss: 0.00377718
[30]	training's binary_logloss: 0.00200666	valid_1's binary_logloss: 0.00371041
[40]	training's binary_logloss: 0.00172191	valid_1's binary_logloss: 0.00367748
[50]

[100]	training's binary_logloss: 0.0254468	valid_1's binary_logloss: 0.0397882
Did not meet early stopping. Best iteration is:
[100]	training's binary_logloss: 0.0254468	valid_1's binary_logloss: 0.0397882
Training until validation scores don't improve for 100 rounds
[10]	training's binary_logloss: 0.0399697	valid_1's binary_logloss: 0.0556162
[20]	training's binary_logloss: 0.0365369	valid_1's binary_logloss: 0.0554575
[30]	training's binary_logloss: 0.0340481	valid_1's binary_logloss: 0.0552789
[40]	training's binary_logloss: 0.0313477	valid_1's binary_logloss: 0.0552318
[50]	training's binary_logloss: 0.0290196	valid_1's binary_logloss: 0.0550693
[60]	training's binary_logloss: 0.0271191	valid_1's binary_logloss: 0.0549527
[70]	training's binary_logloss: 0.0254162	valid_1's binary_logloss: 0.0548915
[80]	training's binary_logloss: 0.0236976	valid_1's binary_logloss: 0.0547564
[90]	training's binary_logloss: 0.0222384	valid_1's binary_logloss: 0.0547635
[100]	training's binary_loglos

[40]	training's binary_logloss: 0.00805768	valid_1's binary_logloss: 0.022613
[50]	training's binary_logloss: 0.00702536	valid_1's binary_logloss: 0.0226118
[60]	training's binary_logloss: 0.00622852	valid_1's binary_logloss: 0.0227123
[70]	training's binary_logloss: 0.00558099	valid_1's binary_logloss: 0.022777
[80]	training's binary_logloss: 0.00503726	valid_1's binary_logloss: 0.0228512
[90]	training's binary_logloss: 0.00462181	valid_1's binary_logloss: 0.02291
[100]	training's binary_logloss: 0.00424561	valid_1's binary_logloss: 0.0230257
Did not meet early stopping. Best iteration is:
[100]	training's binary_logloss: 0.00424561	valid_1's binary_logloss: 0.0230257
Training until validation scores don't improve for 100 rounds
[10]	training's binary_logloss: 0.0153096	valid_1's binary_logloss: 0.019937
[20]	training's binary_logloss: 0.0120121	valid_1's binary_logloss: 0.0199743
[30]	training's binary_logloss: 0.00967945	valid_1's binary_logloss: 0.0199561
[40]	training's binary_log

[100]	training's binary_logloss: 0.00266179	valid_1's binary_logloss: 0.0182921
Did not meet early stopping. Best iteration is:
[100]	training's binary_logloss: 0.00266179	valid_1's binary_logloss: 0.0182921
Training until validation scores don't improve for 100 rounds
[10]	training's binary_logloss: 0.0107992	valid_1's binary_logloss: 0.0138847
[20]	training's binary_logloss: 0.00818163	valid_1's binary_logloss: 0.0139077
[30]	training's binary_logloss: 0.0066106	valid_1's binary_logloss: 0.0138915
[40]	training's binary_logloss: 0.00561869	valid_1's binary_logloss: 0.0139001
[50]	training's binary_logloss: 0.00488881	valid_1's binary_logloss: 0.0139009
[60]	training's binary_logloss: 0.00434241	valid_1's binary_logloss: 0.0139039
[70]	training's binary_logloss: 0.00391194	valid_1's binary_logloss: 0.0139102
[80]	training's binary_logloss: 0.00353933	valid_1's binary_logloss: 0.0139284
[90]	training's binary_logloss: 0.00323232	valid_1's binary_logloss: 0.0139339
[100]	training's bina

...

[20]	training's binary_logloss: 0.00909871	valid_1's binary_logloss: 0.0202118
[30]	training's binary_logloss: 0.00732876	valid_1's binary_logloss: 0.0201666
[40]	training's binary_logloss: 0.00624802	valid_1's binary_logloss: 0.0201843
[50]	training's binary_logloss: 0.00543173	valid_1's binary_logloss: 0.0201389
[60]	training's binary_logloss: 0.00482543	valid_1's binary_logloss: 0.0200934
[70]	training's binary_logloss: 0.00437679	valid_1's binary_logloss: 0.0200942
[80]	training's binary_logloss: 0.00399972	valid_1's binary_logloss: 0.0201054
[90]	training's binary_logloss: 0.00365898	valid_1's binary_logloss: 0.0201455
[100]	training's binary_logloss: 0.00338274	valid_1's binary_logloss: 0.0201776
Did not meet early stopping. Best iteration is:
[100]	training's binary_logloss: 0.00338274	valid_1's binary_logloss: 0.0201776
Training until validation scores don't improve for 100 rounds
[10]	training's binary_logloss: 0.0118435	valid_1's binary_logloss: 0.0175905
[20]	training's bina

[80]	training's binary_logloss: 0.00195059	valid_1's binary_logloss: 0.0141207
[90]	training's binary_logloss: 0.00176571	valid_1's binary_logloss: 0.0142046
[100]	training's binary_logloss: 0.00160682	valid_1's binary_logloss: 0.0142604
Did not meet early stopping. Best iteration is:
[100]	training's binary_logloss: 0.00160682	valid_1's binary_logloss: 0.0142604
Training until validation scores don't improve for 100 rounds
[10]	training's binary_logloss: 0.00658998	valid_1's binary_logloss: 0.0123953
[20]	training's binary_logloss: 0.00477581	valid_1's binary_logloss: 0.0124356
[30]	training's binary_logloss: 0.00388042	valid_1's binary_logloss: 0.0124562
[40]	training's binary_logloss: 0.00331421	valid_1's binary_logloss: 0.0124975
[50]	training's binary_logloss: 0.00292182	valid_1's binary_logloss: 0.0125183
[60]	training's binary_logloss: 0.00262107	valid_1's binary_logloss: 0.012514
[70]	training's binary_logloss: 0.0023317	valid_1's binary_logloss: 0.0125241
[80]	training's binar

[20]	training's binary_logloss: 0.0401301	valid_1's binary_logloss: 0.0453
[30]	training's binary_logloss: 0.0365591	valid_1's binary_logloss: 0.0450723
[40]	training's binary_logloss: 0.0337715	valid_1's binary_logloss: 0.0450484
[50]	training's binary_logloss: 0.0315595	valid_1's binary_logloss: 0.0450026
[60]	training's binary_logloss: 0.0296712	valid_1's binary_logloss: 0.0448446
[70]	training's binary_logloss: 0.0279011	valid_1's binary_logloss: 0.0447462
[80]	training's binary_logloss: 0.0260775	valid_1's binary_logloss: 0.0447012
[90]	training's binary_logloss: 0.0246091	valid_1's binary_logloss: 0.0446725
[100]	training's binary_logloss: 0.0233335	valid_1's binary_logloss: 0.0446207
Did not meet early stopping. Best iteration is:
[100]	training's binary_logloss: 0.0233335	valid_1's binary_logloss: 0.0446207
Training until validation scores don't improve for 100 rounds
[10]	training's binary_logloss: 0.0194793	valid_1's binary_logloss: 0.0272985
[20]	training's binary_logloss: 0

[80]	training's binary_logloss: 0.0131552	valid_1's binary_logloss: 0.0232215
[90]	training's binary_logloss: 0.012254	valid_1's binary_logloss: 0.0231301
[100]	training's binary_logloss: 0.0114117	valid_1's binary_logloss: 0.0230465
Did not meet early stopping. Best iteration is:
[100]	training's binary_logloss: 0.0114117	valid_1's binary_logloss: 0.0230465
Training until validation scores don't improve for 100 rounds
[10]	training's binary_logloss: 0.0232775	valid_1's binary_logloss: 0.0421203
[20]	training's binary_logloss: 0.0198735	valid_1's binary_logloss: 0.0421452
[30]	training's binary_logloss: 0.0169511	valid_1's binary_logloss: 0.0422171
[40]	training's binary_logloss: 0.0150256	valid_1's binary_logloss: 0.0421077
[50]	training's binary_logloss: 0.013317	valid_1's binary_logloss: 0.0420477
[60]	training's binary_logloss: 0.0119306	valid_1's binary_logloss: 0.0420652
[70]	training's binary_logloss: 0.0109165	valid_1's binary_logloss: 0.0420287
[80]	training's binary_logloss: 

Training until validation scores don't improve for 100 rounds
[10]	training's binary_logloss: 0.005371	valid_1's binary_logloss: 0.00796562
[20]	training's binary_logloss: 0.00396328	valid_1's binary_logloss: 0.00793186
[30]	training's binary_logloss: 0.00323765	valid_1's binary_logloss: 0.00789364
[40]	training's binary_logloss: 0.00274449	valid_1's binary_logloss: 0.00785066
[50]	training's binary_logloss: 0.00237749	valid_1's binary_logloss: 0.00781571
[60]	training's binary_logloss: 0.00208566	valid_1's binary_logloss: 0.00778376
[70]	training's binary_logloss: 0.00184772	valid_1's binary_logloss: 0.0077832
[80]	training's binary_logloss: 0.00164219	valid_1's binary_logloss: 0.00777586
[90]	training's binary_logloss: 0.0014676	valid_1's binary_logloss: 0.00779576
[100]	training's binary_logloss: 0.00131592	valid_1's binary_logloss: 0.00780588
Did not meet early stopping. Best iteration is:
[100]	training's binary_logloss: 0.00131592	valid_1's binary_logloss: 0.00780588
Training unt

In [ ]:
scores = []
for target_col in target_cols:
    scores.append(log_loss(train_targets_scored[target_col], oof[target_col]))
print(np.mean(scores))

In [ ]:
if debug == False:
    submission.to_csv("submission.csv", index=False)
    submission.head()

## optuna

In [ ]:
def objective(trial):
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        #'metric': 'rmse',
        'verbosity': -1,
        "seed":42,
        "learning_rate":trial.suggest_loguniform('learning_rate', 0.005, 0.03),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }
    FOLD_NUM = 5
    mskf = MultilabelStratifiedKFold(n_splits=FOLD_NUM, shuffle=True, random_state=42)
    scores = []
    feature_importance_df = pd.DataFrame()

    pred_cv = np.zeros(len(test.index))
    num_round = 10000

    


    for i, (tdx, vdx) in enumerate(kf.split(train_X[selected], train_y)):
        print(f'Fold : {i}')
        X_train, X_valid, y_train, y_valid = train_X[selected].iloc[tdx], train_X[selected].iloc[vdx], train_y.values[tdx], train_y.values[vdx]
        lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=object_cols)
        lgb_valid = lgb.Dataset(X_valid, y_valid, categorical_feature=object_cols)
        model = lgb.train(params, lgb_train, num_boost_round=num_round,
                      valid_names=["train", "valid"], valid_sets=[lgb_train, lgb_valid],
                      early_stopping_rounds=100, verbose_eval=10000)

        va_pred = model.predict(X_valid)
        va_pred[va_pred<0] = 0
        score_ = np.sqrt(mean_squared_log_error(np.expm1(y_valid), np.expm1(va_pred)))
        scores.append(score_)

    return np.mean(scores)

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
# 結果の確認
print('Best trial:')
light_trial = study.best_trial

print('  Value: {}'.format(light_trial.value))

print('  Params: ')

In [ ]:
with open("lightgbmparams.txt", "w") as file:
    for key, value in light_trial.params.items():
       print('    "{}": {},'.format(key, value))
       file.write('"{}": {},'.format(key, value))